# Processing Sensor Data from Kafka with Structured Streaming

The intention of this example is to explore the main aspects of the Structured Streaming API.

We will: 
 - use the Kafka `source` to consume events from the `sensor-raw` topic in Kafka
 - implement the application logic using the Dataset API
 - use the `memory` sink to visualize the data
 - use the `kafka` sink to publish our results to a different topic and make it available downstream.
 - have some fun!  

##Common Definitions
We define a series of parameters common to  the notebook

In [ ]:
val sourceTopic = "sensor-raw"
val targetTopic = "sensor-processed"
val kafkaBootstrapServer = "172.17.0.2:9092" // local
// val kafkaBootstrapServer = "10.2.2.191:1025" // fast-data-ec2

sourceTopic: String = sensor-raw
targetTopic: String = sensor-processed
kafkaBootstrapServer: String = 172.17.0.2:9092


## Read a stream from Kafka
We use the kafka source to subscribe to the `sourceTopic` that contains the raw sensor data.
This results in a streaming dataframe that we use to operate on the underlying data

In [ ]:
val rawData = sparkSession.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", kafkaBootstrapServer)
      .option("subscribe", sourceTopic)
      .option("startingOffsets", "latest")
      .load()

rawData: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]


In [ ]:
rawData.isStreaming

res3: Boolean = true


true

In [ ]:
rawData.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



## Declare the schema of the data in the stream
We need to declare the schema of the data in the stream in order to parse it.

We use a case class to define the schema. It's much more convenient that using the sql types directly.

In [ ]:
case class SensorData(id: String, ts: Long, temp: Double, hum: Double)

defined class SensorData


In [ ]:
import org.apache.spark.sql.Encoders
val schema = Encoders.product[SensorData].schema

import org.apache.spark.sql.Encoders
schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true), StructField(ts,LongType,false), StructField(temp,DoubleType,false), StructField(hum,DoubleType,false))


## Parse the Data
The actual payload is contained in the 'value' field that we get from the kafka topic (see above).
We first need to convert that binary value field to string and then use the `json` support in Spark to transform our incoming data into a structured streaming `Dataset`


In [ ]:
val rawValues = rawData.selectExpr("CAST(value AS STRING)").as[String]
val jsonValues = rawValues.select(from_json($"value", schema) as "record")
val sensorData = jsonValues.select("record.*").as[SensorData]

rawValues: org.apache.spark.sql.Dataset[String] = [value: string]
jsonValues: org.apache.spark.sql.DataFrame = [record: struct<id: string, ts: bigint ... 2 more fields>]
sensorData: org.apache.spark.sql.Dataset[SensorData] = [id: string, ts: bigint ... 2 more fields]


In [ ]:
sensorData.printSchema()

root
 |-- id: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- temp: double (nullable = true)
 |-- hum: double (nullable = true)



## Explore the data stream
To view the streaming data, we will use the `memory` sink and query the resulting table to get samples of the data.

In [ ]:
val visualizationQuery = sensorData.writeStream
  .queryName("visualization")    // this query name will be the SQL table name
  .outputMode("append")
  .format("memory")
  .start()

visualizationQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7d7b2519


## Explore the Data
The `memory` sink creates an in-memory SQL table (like a `tempTable`) that we can query using Spark SQL
The result of the query is a static `Dataframe` that contains a snapshot of the data.

In [ ]:
val sampleDataset = sparkSession.sql("select * from visualization")


sampleDataset: org.apache.spark.sql.DataFrame = [id: string, ts: bigint ... 2 more fields]


In [ ]:
// This is a static Dataset!
sampleDataset.isStreaming

res14: Boolean = false


false

### Our dataset is backed by the streaming data, it will update each time we execute an action, delivering the latest data.

In [ ]:
sampleDataset.count

res16: Long = 88


88

In [ ]:
sampleDataset.count

res18: Long = 144


144

## Visualize the Data
We will make a custom live update by querying the stream every so often for the latest updates

In [ ]:
val dummy = Seq((System.currentTimeMillis, 0.1), (System.currentTimeMillis, 0.1))

val chart = CustomPlotlyChart(dummy,
                  layout=s"{title: 'sensor data sample'}",
                  dataOptions="""{type: 'line'}""",
                  dataSources="{x: '_1', y: '_2' }")

dummy: Seq[(Long, Double)] = List((1542186313065,0.1), (1542186313066,0.1))
chart: notebook.front.widgets.charts.CustomPlotlyChart[Seq[(Long, Double)]] = <CustomPlotlyChart widget>


## Async update of our visualization
We will use a plain old Thread to run a recurrent query on our in-memory table and update the chart accordingly.


In [ ]:
@volatile var running = true

running: Boolean = true


In [ ]:
import scala.concurrent.duration._
import scala.annotation.tailrec

val updater = new Thread() {
  @tailrec
  def visualize(): Unit = {
    val lastMinute = System.currentTimeMillis - 1.minute.toMillis
    val data = sampleDataset.where($"ts" > lastMinute and $"id" === "devoxx").as[SensorData]
                            .map{case SensorData(id, ts, value, level) => (ts/1000%3600, value)}.collect
    chart.applyOn(data)
    if (running) {
      Thread.sleep(1.second.toMillis)
      visualize()
    } else ()
  } 
  
  override def run() {
    visualize()
  }
}.start()


import scala.concurrent.duration._
import scala.annotation.tailrec
updater: Unit = ()


In [ ]:
chart

res23: notebook.front.widgets.charts.CustomPlotlyChart[Seq[(Long, Double)]] = <CustomPlotlyChart widget>


entries total

In [ ]:
// visualizationQuery.stop()
// running = false

# Improve the data with sliding windows

In [ ]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._


In [ ]:
val toSeconds = udf((ts:Long) => ts/1000)
val sensorMovingAverage = sensorData.withColumn("timestamp", toSeconds($"ts").cast(TimestampType))
                                          .withWatermark("timestamp", "30 seconds")
                                          .groupBy($"id", window($"timestamp", "30 seconds", "10 seconds"))
                                          .agg(avg($"temp"))

toSeconds: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,LongType,Some(List(LongType)))
sensorMovingAverage: org.apache.spark.sql.DataFrame = [id: string, window: struct<start: timestamp, end: timestamp> ... 1 more field]


In [ ]:
sensorMovingAverage.printSchema

root
 |-- id: string (nullable = true)
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- avg(temp): double (nullable = true)



In [ ]:
val windowedSensorQuery = sensorMovingAverage.writeStream
  .queryName("movingAverage")    // this query name will be the table name
  .outputMode("append")  
  .format("memory")
  .start()

windowedSensorQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7917a017


### Get the data from the in-memory table

In [ ]:
val movingAvgDF = sparkSession.sql("select * from movingAverage")

<console>:121: error: type mismatch;
 found   : $iw
 required: $iw
val $iw = new $iw
    ^


In [ ]:
movingAvgDF

<console>:70: error: not found: value movingAvgDF
       movingAvgDF
       ^


### Chart the Moving Average Data

In [ ]:
import org.apache.spark.sql.functions._
val lastMinute: Long = System.currentTimeMillis/1000 - 5.minute.toSeconds
val mAvgSample = movingAvgDF.select($"window.start".cast(LongType) as "timestamp", $"avg(temp)" as "temp")
                   .where($"timestamp" > lastMinute and $"id" === "devoxx")
                   .orderBy($"timestamp")
                   .as[(Long, Double)]
                   .collect().map{case (ts, v) => (ts  % 3600,v)}


CustomPlotlyChart(mAvgSample,
                  layout=s"{title: 'moving average sensor data'}",
                  dataOptions="""{type: 'line'}""",
                  dataSources="{x: '_1', y: '_2'}")

<console>:70: error: not found: value movingAvgDF
val mAvgSample = movingAvgDF.select($"window.start".cast(LongType) as "timestamp", $"avg(temp)" as "temp")
                 ^
<console>:74: error: value % is not a member of Any
                   .collect().map{case (ts, v) => (ts  % 3600,v)}
                                                       ^


In [ ]:
// stop the ancilliary visualization queries
windowedSensorQuery.stop()
visualizationQuery.stop()
running = false

running: Boolean = false


## Write our moving average data to our `sensor-clean` topic

In [ ]:
// First we prepare the schema to comply with the (key, value) model of Kafka
val kafkaFormat = sensorMovingAverage
.select($"id", $"window.start".cast(LongType) as "timestamp", $"avg(temp)" as "temp")
.select($"id" as "key", to_json(struct($"id", $"timestamp", $"temp")) as "value")

kafkaFormat: org.apache.spark.sql.DataFrame = [key: string, value: string]


In [ ]:
val kafkaWriterQuery = kafkaFormat.writeStream
  .queryName("kafkaWriter") 
  .outputMode("append") 
  .format("kafka")
  .option("kafka.bootstrap.servers", kafkaBootstrapServer)
  .option("topic", targetTopic)
  .option("checkpointLocation", "/tmp/spark/checkpoint4")
  .option("failOnDataLoss", "false")
  .start()

kafkaWriterQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@51b8963a


## View Progress

In [ ]:
val progress = kafkaWriterQuery.recentProgress

progress: Array[org.apache.spark.sql.streaming.StreamingQueryProgress] =
Array({
  "id" : "34e7f374-6a2f-4b02-992b-eb329d52ed45",
  "runId" : "26ffdf13-b583-48fd-90f0-d507e51f16ab",
  "name" : "kafkaWriter",
  "timestamp" : "2018-11-14T09:05:26.155Z",
  "batchId" : 2218,
  "numInputRows" : 63,
  "processedRowsPerSecond" : 14.02493321460374,
  "durationMs" : {
    "addBatch" : 4390,
    "getBatch" : 9,
    "queryPlanning" : 62,
    "triggerExecution" : 4492
  },
  "eventTime" : {
    "avg" : "2018-11-14T00:24:13.666Z",
    "max" : "2018-11-14T00:24:15.000Z",
    "min" : "2018-11-14T00:24:13.000Z",
    "watermark" : "2018-11-14T00:23:43.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 4518,
    "numRowsUpdated" : 186,
    "memoryUsedBytes" : 1323799
  } ],
  "sources" : [ {
    "d...

In [ ]:
progress.map(entry  => (entry.inputRowsPerSecond, entry.processedRowsPerSecond))

res41: Array[(Double, Double)] = Array()


empty array